<a href="https://colab.research.google.com/github/WolfgangNS/GTF-to-matrix/blob/main/Renn_Lab_TPM_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#courtesy of ChatGPT

#data pipeline:
#https://chatgpt.com/c/4c7769c8-c3a3-4562-b962-9ff1357a9bdd

#data download:
#https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5769613
#GSM5769613_SR02_merged.gtf.gz

#-----
#upload .gtf file to /content directory

### Download mappings (BioMart)

In [ ]:
#made into Python variable so the query is easier to read :)
#https://useast.ensembl.org/biomart/martview

import pandas as pd
import urllib.parse

query = (
    '<?xml version="1.0" encoding="UTF-8"?>'
    '<!DOCTYPE Query>'
    '<Query  virtualSchemaName = "default" formatter = "TSV" header = "0" uniqueRows = "0" count = "" datasetConfigVersion = "0.6" >'
    '<Dataset name = "hburtoni_gene_ensembl" interface = "default" >'
    '<Attribute name = "hburtoni_paralog_ensembl_gene" />'
    '<Attribute name = "oniloticus_homolog_ensembl_gene" />'
    '</Dataset>'
    '</Query>'
)
encoded_query = urllib.parse.quote(query)
wget_command = f"""
wget -q --show-progress -O genes.tsv http://www.ensembl.org/biomart/martservice?query={encoded_query}
"""

print(wget_command)
!{wget_command}

orthologs = pd.read_csv('genes.tsv',
                        sep='\t',
                        names=['hburtoni_paralog_ensembl_gene',
                                'oniloticus_homolog_ensembl_gene'])
orthologs.head(10)


wget -q --show-progress -O genes.tsv http://www.ensembl.org/biomart/martservice?query=%3C%3Fxml%20version%3D%221.0%22%20encoding%3D%22UTF-8%22%3F%3E%3C%21DOCTYPE%20Query%3E%3CQuery%20%20virtualSchemaName%20%3D%20%22default%22%20formatter%20%3D%20%22TSV%22%20header%20%3D%20%220%22%20uniqueRows%20%3D%20%220%22%20count%20%3D%20%22%22%20datasetConfigVersion%20%3D%20%220.6%22%20%3E%3CDataset%20name%20%3D%20%22hburtoni_gene_ensembl%22%20interface%20%3D%20%22default%22%20%3E%3CAttribute%20name%20%3D%20%22hburtoni_paralog_ensembl_gene%22%20/%3E%3CAttribute%20name%20%3D%20%22oniloticus_homolog_ensembl_gene%22%20/%3E%3C/Dataset%3E%3C/Query%3E

genes.tsv               [      <=>           ]  13.22M   913KB/s    in 17s     


,hburtoni_paralog_ensembl_gene,oniloticus_homolog_ensembl_gene
0,NaN,NaN
1,ENSHBUG00000023832,ENSONIG00000040832
2,ENSHBUG00000019623,ENSONIG00000040832
3,ENSHBUG00000016228,ENSONIG00000040832
4,ENSHBUG00000005138,ENSONIG00000040832
5,ENSHBUG00000016083,ENSONIG00000040832
6,ENSHBUG00000019072,ENSONIG00000040832
7,ENSHBUG00000008100,ENSONIG00000040832
8,ENSHBUG00000013528,ENSONIG00000040832
9,ENSHBUG00000012627,ENSONIG00000040832


In [ ]:
#Again, but different
query = (
    '<?xml version="1.0" encoding="UTF-8"?>'
    '<!DOCTYPE Query>'
    '<Query  virtualSchemaName = "default" formatter = "TSV" header = "0" uniqueRows = "0" count = "" datasetConfigVersion = "0.6" >'
    '<Dataset name = "hburtoni_gene_ensembl" interface = "default" >'
    '<Attribute name = "ensembl_gene_id" />'
    '<Attribute name = "entrezgene_accession" />'
    '</Dataset>'
    '</Query>'
)
encoded_query = urllib.parse.quote(query)
wget_command = f"""
wget -q --show-progress -O genes2.tsv http://www.ensembl.org/biomart/martservice?query={encoded_query}
"""

print(wget_command)
!{wget_command}

ens_to_enz = pd.read_csv('genes2.tsv',
                        sep='\t',
                        names=['ensembl_gene_id',
                                'entrezgene_accession'])
ens_to_enz.head(10)


wget -q --show-progress -O genes2.tsv http://www.ensembl.org/biomart/martservice?query=%3C%3Fxml%20version%3D%221.0%22%20encoding%3D%22UTF-8%22%3F%3E%3C%21DOCTYPE%20Query%3E%3CQuery%20%20virtualSchemaName%20%3D%20%22default%22%20formatter%20%3D%20%22TSV%22%20header%20%3D%20%220%22%20uniqueRows%20%3D%20%220%22%20count%20%3D%20%22%22%20datasetConfigVersion%20%3D%20%220.6%22%20%3E%3CDataset%20name%20%3D%20%22hburtoni_gene_ensembl%22%20interface%20%3D%20%22default%22%20%3E%3CAttribute%20name%20%3D%20%22ensembl_gene_id%22%20/%3E%3CAttribute%20name%20%3D%20%22entrezgene_accession%22%20/%3E%3C/Dataset%3E%3C/Query%3E

genes2.tsv              [                <=> ] 622.62K  99.5KB/s    in 6.3s    


,ensembl_gene_id,entrezgene_accession
0,ENSHBUG00000023839,NaN
1,ENSHBUG00000000727,LOC102288583
2,ENSHBUG00000023838,hps1
3,ENSHBUG00000023837,LOC102292000
4,ENSHBUG00000023836,LOC102301706
5,ENSHBUG00000023835,slc25a29
6,ENSHBUG00000023834,LOC102298492
7,ENSHBUG00000023833,agap2
8,ENSHBUG00000023832,LOC102291978
9,ENSHBUG00000023831,LOC106633135


In [ ]:
joined_df = pd.merge(ens_to_enz, orthologs, left_on='ensembl_gene_id', right_on='hburtoni_paralog_ensembl_gene', how='inner')
#TO-DO: I might need to change this to outer join

joined_df = joined_df.dropna()
joined_df = joined_df.drop(columns=['hburtoni_paralog_ensembl_gene'])

before_rows = len(ens_to_enz)
after_rows = len(joined_df)

print("Number of rows before joining:", before_rows)
print("Number of rows after joining and removing NaNs:", after_rows)
print("Preview of the resulting DataFrame:")
print(joined_df.head())

Number of rows before joining: 23922
Number of rows after joining and removing NaNs: 261369
Preview of the resulting DataFrame:
      ensembl_gene_id entrezgene_accession oniloticus_homolog_ensembl_gene
0  ENSHBUG00000000727         LOC102288583              ENSONIG00000040832
1  ENSHBUG00000000727         LOC102288583              ENSONIG00000035632
2  ENSHBUG00000000727         LOC102288583              ENSONIG00000008077
3  ENSHBUG00000000727         LOC102288583              ENSONIG00000041447
4  ENSHBUG00000000727         LOC102288583              ENSONIG00000013172


In [ ]:
joined_df.to_csv("/content/entrez-ensembl-mappings.tsv", sep='\t')

### Download .tar (.gtf files inside)

In [ ]:
#https://ftp.ncbi.nlm.nih.gov/geo/series/GSE192nnn/GSE192958/suppl/
!wget https://ftp.ncbi.nlm.nih.gov/geo/series/GSE192nnn/GSE192958/suppl/GSE192958_RAW.tar
!tar -xvf GSE192958_RAW.tar -k

### Create gene count matrix from samples (HTSeq)

In [ ]:
!pip install HTSeq scanpy

In [ ]:
#15m for code below

In [ ]:
import os
import HTSeq
import numpy as np
import pandas as pd

# Directory containing .gtf files
directory = "/content"

# Function to extract gene expression counts from a single .gtf file
def extract_counts(gtf_file):
    counts = {}
    features = HTSeq.GFF_Reader(gtf_file)
    for feature in features:
        if feature.type == "transcript":
            gene_name = feature.attr.get("gene_name")
            ref_gene_name = feature.attr.get("ref_gene_name")
            tpm = feature.attr.get("TPM")  # Get TPM attribute
            if tpm is not None:
                tpm_value = float(tpm)
                if gene_name is not None:
                    counts[gene_name] = counts.get(gene_name, 0) + tpm_value
                elif ref_gene_name is not None:
                    counts[ref_gene_name] = counts.get(ref_gene_name, 0) + tpm_value
    return counts

# Iterate over .gtf files in the directory
all_counts = {}
for filename in os.listdir(directory):
    if filename.endswith(".gtf"):
        filepath = os.path.join(directory, filename)
        sample_name = os.path.splitext(filename)[0]
        counts = extract_counts(filepath)
        all_counts[sample_name] = counts

# Convert counts to DataFrame and save as TSV
gene_names = sorted(set().union(*(counts.keys() for counts in all_counts.values())))

sample_names = list(all_counts.keys())
count_matrix = np.zeros((len(sample_names), len(gene_names)), dtype=int)
for i, sample_name in enumerate(sample_names):
    counts = all_counts[sample_name]
    for j, gene_name in enumerate(gene_names):
        count_matrix[i, j] = counts.get(gene_name, 0)

count_df = pd.DataFrame(count_matrix, index=sample_names, columns=gene_names)
count_df.to_csv("/content/output.tsv", sep='\t')

In [ ]:
count_df.head()

In [ ]:
!head output.tsv

In [ ]:
##Hash table lookup for ENSEMBL identifiers

gene_mapping = dict(zip(joined_df["entrezgene_accession"], joined_df["oniloticus_homolog_ensembl_gene"]))
# count_df = pd.read_csv("/content/output.tsv", sep='\t', index_col=0)
count_df.columns = [gene_mapping.get(col, col) for col in count_df.columns]
transposed_count_df = count_df.transpose()
# transposed_count_df.to_csv("/content/transposed_output.tsv", sep='\t')

In [ ]:
#Handle duplicated rows by summing them together

# Find duplicated row names
duplicated_rows = transposed_count_df.index[transposed_count_df.index.duplicated()]

# Sum together the contents of the cells for duplicated rows
for row_name in duplicated_rows:
    duplicated_row = transposed_count_df.loc[row_name]
    summed_row = duplicated_row.groupby(level=0).sum()
    transposed_count_df.loc[row_name] = summed_row

# Drop duplicate rows
transposed_count_df = transposed_count_df.loc[~transposed_count_df.index.duplicated()]


In [ ]:
transposed_count_df.sort_values(by=['GSM5769615_SR05_merged'], ascending=False).head()

,GSM5769620_SR11_merged,GSM5769626_SR17_merged,GSM5769630_SR21_merged,GSM5769655_SR47_merged,GSM5769647_SR39_merged,GSM5769650_SR42_merged,GSM5769644_SR35_merged,GSM5769651_SR43_merged,GSM5769648_SR40_merged,GSM5769642_SR33_merged,...,GSM5769637_SR28_merged,GSM5769635_SR26_merged,GSM5769616_SR06_merged,GSM5769614_SR03_merged,GSM5769639_SR30_merged,GSM5769623_SR14_merged,GSM5769652_SR44_merged,GSM5769649_SR41_merged,GSM5769638_SR29_merged,GSM5769622_SR13_merged
ENSONIG00000001129,-5329097633061404672,2256264999125123072,-7382176456786313216,-5925084955959885824,-6238561287316439040,6989375309288046592,5704504644047208448,-2163368979681247232,7302851640644599808,-8463860946626936832,...,4764075649977548800,-5783829460188921856,5563249148276244480,-8322605450855972864,-5783829460188921856,2256264999125123072,-7037734785615134720,-737242818669445120,971394333884284928,-5956050295774511104
ENSONIG00000001184,1869529919568370027,-943823577690462157,-5259821628494995885,4583801790761317631,8672761142278252843,-1654380813794127542,-5440433335899251239,756984227987008547,-1066118699698018318,-4729876099795585854,...,2882993985083846927,-1229178862374759866,-4549264392391330500,8369854312866441328,-6756804230826539654,-5987930409326175076,8876586345624179778,2498557074333664638,-4124062440971962824,5958489271085305239
ENSONIG00000042451,-8863182617562100722,-5754057652588050633,-6940954517218146521,-3204488750134822392,6489388888301908946,-5372602511944961421,6366446057568816532,8916014960022044773,1390251083395452464,8384037943073543036,...,1335092992250812242,2466831765736267908,-6381398454697324673,6815685937800357936,-4117920907726413341,2303683240987043413,7907219017269681521,8384037943073543036,6938628768533450350,-55158091144640222
ENSONIG00000004031,-7412778459228798976,-3634815588910497792,8491148148768505856,-515596934382092288,925406037658304512,-8524409323374247936,8687189345729249280,2284065982724964352,2160551442264358912,6226714906990739456,...,-299922996474150912,-8318551755939905536,7575558481583800320,3436868360357281792,-7124577864820719616,4909226475410948096,-556768447868960768,1501807226474463232,7832403932978610176,-4046530723779182592
ENSONIG00000005674,-2262141455386447985,-2446690272923345516,8810936782178011034,2997574437090435228,-3909120185649904200,-2508206545435644693,5234556925783242966,72714611637317860,3993033136412240743,-4622156489103854077,...,-1266682756064642470,8970326633021268318,-5961553856805814407,6635470565997502473,-1045776632709086009,7913351661187163626,-9009446227283530129,-1758812936163035886,-676678997635290947,-4437607671566956546


In [ ]:
transposed_count_df.to_csv("/content/transposed_output.tsv", sep='\t')

### Phenotype data

In [ ]:
!wget https://ftp.ncbi.nlm.nih.gov/geo/series/GSE192nnn/GSE192958/miniml/GSE192958_family.xml.tgz
!tar -zxvf GSE192958_family.xml.tgz

In [ ]:
import xml.etree.ElementTree as ET
tree = ET.parse('GSE192958_family.xml')
root = tree.getroot()
with open('attributes.tsv', 'w') as tsvfile:
    tsvfile.write('GSM_id\tsamplename\n')
    for sample in root.findall('{http://www.ncbi.nlm.nih.gov/geo/info/MINiML}Sample'):
        sample_id = sample.attrib['iid']
        for subsample in sample.findall('{http://www.ncbi.nlm.nih.gov/geo/info/MINiML}Title'):
            # print(subsample.text)
            # print(sample.attrib['iid'])
            GSM_id = subsample.text
            samplename = sample.attrib['iid']
            tsvfile.write(f'{samplename}\t{GSM_id}\n')


In [ ]:
!head -n 10 attributes.tsv

### (Miscellaneous)

In [ ]:
#Briliant solution: https://unix.stackexchange.com/a/22624
!tail -n 420 GSE192958_series_matrix.txt \
  | head -30